In [98]:
import datetime
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import datetime
from DOSlib.util import connect2DB

In [154]:
# instance range
first_obsday = 20210515
last_obsday = 20241214
# roles
#roles = ['GUIDE0', 'GUIDE2', 'GUIDE3','GUIDE5','GUIDE7', 'GUIDE8','FOCUS1', 'FOCUS4', 'FOCUS6', 'FOCUS9', 'SKYCAM0', 'SKYCAM1']
roles = ['OCS']  #'PETAL%d' %x for x in range(0,10)]
#roles = ['SPECTCON%d' %x for x in range(10)]
#roles = ['GUDE8'] #, 'GUIDE7', 'GUIDE3'] #roles=['GUIDERIB', 'FOCUSIB', 'SKYIB']
#for i in range(10):
#    for j in ['Z','R','B']:
#        roles.append(f'CCDS{i}{j}')
#for i in range(10):
#    roles.append(f'PETAL{i}')
print('Using these roles', roles)
# search string
search_strings = ['NFS constraints']
remember_strings = [] #['execute_moves: execute_moves: Done'] #['Moving telescope to:'] #['flavor zero']

Using these roles ['OCS']


In [155]:
# line filter functions
line_filter = None

def select_line_and_size(line):
    if ".py:" in line:
        lineno = int(line.split('.py:')[1].split(':')[0])
    if 'size=' in line:
        size = line.split('size=')[1].split(',')[0]
        size, unit = size.split()
        if unit == 'MiB':
            size = float(size)*1024*1024
        elif unit == 'KiB':
            size = float(size)*1024
        else:
            size = float(size)
    return (lineno, size)

def excessive_processing_time(line, remembered):
    id = None
    role = None
    exptime = None
    proctime = None
    id = remembered.split('(')[1].split(')')[0]
    role = remembered.split('-')[1].split(' ')[0]
    exptime = remembered.split('exptime ')[-1].split(',')[0]
    proctime = line.split('processing time: ')[-1]
    proctime = proctime.strip('\n')
    return int(id), role, float(exptime), float(proctime)

def movetable_time(line):
    mt = line.split('+')[-1].split(')')[0].strip()
    id = int(line.split('-')[1].replace('PC0','').split('[')[0].strip())
    return int(id), f'PC0{id}', float(mt)

def max_movetime(line):
    print(line)
    mt = line.split(' ')[-1].strip()
    print(mt)
    id = int(line.split('-')[1].replace('PC0','').split('[')[0].strip())
    return int(id), f'PC0{id}', float(mt)

def time_difference(line, remembered):
    id = None
    role = None
    starttime = None
    stoptime = None
    id = remembered.split('(')[1].split(')')[0]
    starttime = remembered.split('[INFO   ] ')[-1].split('Z')[0]
    checkid = line.split('(')[1].split(')')[0]
    if id != checkid:
        print(f'Exposure number mismatch {id}, {checkid}')
        return None
    stoptime = line.split('[INFO   ] ')[-1].split('Z')[0]   
    startdt = datetime.datetime.strptime(starttime,'%Y-%m-%d %H:%M:%S')
    stopdt = datetime.datetime.strptime(stoptime,'%Y-%m-%d %H:%M:%S')
    slewtime = (stopdt-startdt).seconds
    return int(id), float(slewtime)

#line_filter_name = 'excessive_processing_time'
#line_filter = excessive_processing_time #select_line_and_size
line_filter_name = 'None'
line_filter = None
#line_filter_name='time_difference'
#line_filter = time_difference
#line_filter_name='movetable_time'
#line_filter = movetable_time
#line_filter_name='max_movetime'
#line_filter = max_movetime

In [113]:
# read performance telemetry table to get start and end days per day
conn, cursor = connect2DB(search_path='telemetry')

In [114]:
sql=f"""SELECT distinct obsday, dusk, dawn from performance_night order by obsday asc"""
cursor.execute(sql)
results = cursor.fetchall()
nights_df = pd.DataFrame(results)
nights_df.columns = ['obsday','dusk','dawn']
conn.close()

In [156]:
# Create instances list
instances = []
for i in range(first_obsday, last_obsday+1):
    instances.append(f'desi_{i}')
print(instances)

['desi_20210515', 'desi_20210516', 'desi_20210517', 'desi_20210518', 'desi_20210519', 'desi_20210520', 'desi_20210521', 'desi_20210522', 'desi_20210523', 'desi_20210524', 'desi_20210525', 'desi_20210526', 'desi_20210527', 'desi_20210528', 'desi_20210529', 'desi_20210530', 'desi_20210531', 'desi_20210532', 'desi_20210533', 'desi_20210534', 'desi_20210535', 'desi_20210536', 'desi_20210537', 'desi_20210538', 'desi_20210539', 'desi_20210540', 'desi_20210541', 'desi_20210542', 'desi_20210543', 'desi_20210544', 'desi_20210545', 'desi_20210546', 'desi_20210547', 'desi_20210548', 'desi_20210549', 'desi_20210550', 'desi_20210551', 'desi_20210552', 'desi_20210553', 'desi_20210554', 'desi_20210555', 'desi_20210556', 'desi_20210557', 'desi_20210558', 'desi_20210559', 'desi_20210560', 'desi_20210561', 'desi_20210562', 'desi_20210563', 'desi_20210564', 'desi_20210565', 'desi_20210566', 'desi_20210567', 'desi_20210568', 'desi_20210569', 'desi_20210570', 'desi_20210571', 'desi_20210572', 'desi_2021057

In [157]:
# Scan directories
results = []
for instance in instances:
    dir = os.path.join('/data/msdos/dos_home/instances/', instance, 'logs')
    try:
        files = os.listdir(dir)
    except:
        continue
    for r in roles:
        for f in files:
            if 'current' in f:
                continue
            if r in f:
                filename = os.path.join('/data/msdos/dos_home/instances/',instance, 'logs', f)
                with open(filename,'r') as logfile:
                    try:
                        lines = logfile.readlines()
                    except:
                            continue
                    remembered = True #None
                    for l in lines:
                        if remembered is None:
                            for t in remember_strings:
                                if t in l:
                                    remembered = l
                        else:
                            for t in search_strings:
                                if t in l:
                                    if line_filter is not None:
                                        if line_filter_name == 'time_difference':
                                            retcode = line_filter(l, remembered)
                                            if not isinstance(retcode, (list, tuple)):
                                                continue
                                            results.append({'expid':retcode[0], 'slewtime':retcode[1]})
                                            remembered = None
                                        elif line_filter_name == 'excessive_processing_time':
                                            id, role, exptime, proctime = line_filter(l, remembered)
                                            results.append({'file':f,'expid':id, 'role':role,'exptime':exptime, 'proctime':proctime})
                                            remembered = None 
                                        elif line_filter_name in ['max_movetime','movetable_time']:
                                            id, role, mt = line_filter(l)
                                            results.append({'file':f,'id':id, 'role':role,'movetable':mt})
                                        else:
                                            lineno, result = line_filter(l)
                                            results.append((f, {'lineno':lineno, 'result':result}))
                                    else:
                                        if remember_strings != []:
                                            if remembered is not None:
                                                results.append((f, remembered))
                                                remembered = None
                                                results.append((f,l))
                                        else:
                                            results.append((f,l))
                                                       

In [164]:
for i in range(2):
    print(results[i])


('desi_20210515-OCS-051521142110.log', "desi_20210515-OCS [INFO   ] 2021-05-16 03:36:03Z nfs_set_constraints: NFS constraints updated: {'azrange': [-90, 90]}\n")
('desi_20210516-OCS-051621144935.log', "desi_20210516-OCS [INFO   ] 2021-05-17 03:08:25Z nfs_set_constraints: NFS constraints updated: {'azrange': [-90, 90]}\n")


In [160]:
# Analyze results
wind = []
last_obsday=20210514
last_time="2021-05-14 09:00:00Z"
wind.append({'obsday':last_obsday, 'time_recorded':last_time,'elrange':[None,None],'azrange':[None,None]})
for i in results:
    if not isinstance(i,tuple):
        continue
    w = {}
    instance = i[1].split('-OCS [')[0]
    w['obsday'] = int(instance.split('_')[-1])
    w['time_recorded'] = i[1].split('] ')[-1].split(' nfs_')[0]
    if w['obsday'] != last_obsday:
        obs=str(w['obsday'])
        t = datetime.date(int(obs[0:4]),int(obs[4:6]),int(obs[6:8]))
        tt=t+datetime.timedelta(days=1)
        ts = t.strftime('%Y-%m-%d')
        tts = tt.strftime('%Y-%m-%d')
        # get dusk and dawn from nights_df (see below)
        night = nights_df[nights_df['obsday']==obs]
        try:
            dusk = night.iloc[0].dusk
            dawn = night.iloc[0].dawn
            ts = ts + ' ' + dusk + 'Z'
            tts = tts + ' ' + dawn + 'Z'
            wind.append({'obsday':int(obs),'time_recorded':ts, 'elrange':[None,None],'azrange':[None,None]})
            wind.append({'obsday':int(obs),'time_recorded':tts,'elrange':[None,None],'azrange':[None,None]})
        except:
            pass
        last_obsday = w['obsday']
    if 'cleared' in i[1]:
        w['elrange'] = [None, None]
        w['azrange'] = [None, None]
    else:
        for r in ['azrange', 'elrange']:
            if not r in i[1]:
                w[r] = [None, None]
            else:
                el = i[1].split(r)[-1].split(']')[0].split(': ')[-1]
                el = el.replace('[','').split(', ')
                try:
                    min = float(el[0])
                except:
                    min = None
                try:
                    max = float(el[1])
                except:
                    max = None
                w[r] = [min, max]
    wind.append(w)
print('Done')

Done


In [162]:
winddf = pd.DataFrame(wind)
winddf = winddf.sort_values('time_recorded')
winddf

,obsday,time_recorded,elrange,azrange
0,20210514,2021-05-14 09:00:00Z,"[None, None]","[None, None]"
1,20210515,2021-05-15 20:15:34Z,"[None, None]","[None, None]"
3,20210515,2021-05-16 03:36:03Z,"[None, None]","[-90.0, 90.0]"
2,20210515,2021-05-16 04:28:50Z,"[None, None]","[None, None]"
4,20210516,2021-05-16 20:17:16Z,"[None, None]","[None, None]"
...,...,...,...,...
924,20241212,2024-12-13 06:31:05Z,"[None, None]","[None, None]"
929,20241214,2024-12-14 18:10:42Z,"[None, None]","[None, None]"
931,20241214,2024-12-15 02:39:10Z,"[None, None]","[-179.0, -1.0]"
930,20241214,2024-12-15 06:32:19Z,"[None, None]","[None, None]"


In [163]:
# save to disk
filename = '/data/msdos/wind_constraints.pkl'
winddf.to_pickle(filename)
filename = '/data/msdos/wind_constraints.csv'
winddf.to_csv(filename)